In [1]:
import numpy as np
import pandas as pd

In [2]:
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')

In [4]:
movies = movies.merge(credits,on='title')

In [5]:
movies = movies[['movie_id','title','overview','genres','keywords','cast','crew']]

In [6]:
movies.dropna(inplace=True)

In [9]:
movies.duplicated().sum()

0

In [14]:

def convert(obj):
    L = []
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L

In [16]:
movies['genres'] = movies['genres'].apply(convert)

In [17]:
movies['keywords'] = movies['keywords'].apply(convert)

In [18]:
def convert3(obj):
    L = []
    counter = 0
    for i in ast.literal_eval(obj):
        if counter != 3:
            L.append(i['name'])
            counter += 1
        else:
            break
    return L

In [19]:
movies['cast'] = movies['cast'].apply(convert3)

In [20]:
def fetch_director(obj):
    L = []
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            L.append(i['name'])
            break
    return L

In [21]:
movies['crew'] = movies['crew'].apply(fetch_director)

In [22]:
movies['overview'] = movies['overview'].apply(lambda x:x.split())

In [23]:
movies['genres'] = movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast'] = movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies['crew'] = movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])

In [24]:
movies['tags'] = movies['overview']+movies['genres']+movies['keywords']+movies['cast']+movies['crew']

In [25]:
new_df = movies[['movie_id','title','tags']]

In [26]:
new_df['tags'] = new_df['tags'].apply(lambda x:" ".join(x))

C:\Users\veena\AppData\Local\Temp\ipykernel_10972\3089450492.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x:" ".join(x))


In [28]:
new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())

C:\Users\veena\AppData\Local\Temp\ipykernel_10972\3214958533.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())


In [29]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000, stop_words='english')


In [30]:
vectors = cv.fit_transform(new_df['tags']).toarray()

In [31]:
import nltk

In [32]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [33]:
def stem(text):
    y = []
    
    for i in text.split():
        y.append(ps.stem(i))
    
    return " ".join(y)
    

In [35]:
new_df['tags'] = new_df['tags'].apply(stem)

C:\Users\veena\AppData\Local\Temp\ipykernel_10972\3213734980.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(stem)


In [36]:
ps.stem('danced')

'danc'

In [37]:
from sklearn.metrics.pairwise import cosine_similarity

In [39]:
similarity = cosine_similarity(vectors)

In [45]:
def recommend(movie):
    movie_index = new_df[new_df['title'] == movie].index[0]
    distances = similarity[movie_index]
    movies_list = sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]
    for i in movies_list:
        print(new_df.iloc[i[0]].title)

In [46]:
recommend('Avatar')

Titan A.E.
Small Soldiers
Independence Day
Ender's Game
Aliens vs Predator: Requiem


In [47]:
import pickle
pickle.dump(new_df.to_dict(),open('movie_dict.pkl','wb'))


In [48]:
pickle.dump(similarity, open('similarity.pkl','wb'))